In [2]:
import numpy as np
import pandas as pd

In [3]:
# Uploading the datasets
train = pd.read_csv("train.csv")
store = pd.read_csv("store.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Converting Date into datetime
train['Date'] = pd.to_datetime(train['Date'])

In [5]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [6]:
# Extracting Month and Year attribute
train['Month'] = train['Date'].apply(lambda x: x.month)
train['Year'] = train['Date'].apply(lambda x: x.year)

In [7]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Month,Year
0,1,5,2015-07-31,5263,555,1,1,0,1,7,2015
1,2,5,2015-07-31,6064,625,1,1,0,1,7,2015
2,3,5,2015-07-31,8314,821,1,1,0,1,7,2015
3,4,5,2015-07-31,13995,1498,1,1,0,1,7,2015
4,5,5,2015-07-31,4822,559,1,1,0,1,7,2015


In [8]:
# List all the unique values of month, year and store
storeList = train.Store.unique()
monthList = train.Month.unique()
yearList = train.Year.unique()
columns=['Store', 'Month', 'Year', 'Open', 'Promo', 'Sales']

In [9]:
# Creating new dataframe
final = pd.DataFrame(columns=['Store', 'Month', 'Year', 'Open', 'Promo', 'Sales'])

In [11]:
# For every store aggregate number of open days, promo days and sales for each month
for store in storeList:
    for month in monthList:
        for year in yearList:
            subset = train.loc[lambda df: np.logical_and(np.logical_and(df.Store == store, df.Month == month), df.Year == year)]
            data = {'Store': store,
                    'Month': month,
                    'Year': year,
                    'Open': subset.Open.sum(),
                    'Promo': subset.Promo.sum(),
                    'Sales': subset.Sales.sum()}
            final = final.append(data, ignore_index=True)

In [12]:
for col in columns:
    final[col] = final[col].astype(int)

In [31]:
final['PromoFrequencyRatio'] = final['Promo']/final['Open']

In [48]:
# Combining the 2 datasets
store = pd.read_csv("store.csv")
combine = pd.merge(final, store, on='Store')

In [35]:
combine.head()

,Store,Month,Year,Open,Promo,Sales,PromoFrequencyRatio,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,7,2015,27,13,121266,0.481481,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,7,2014,27,13,122604,0.481481,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,7,2013,27,13,128032,0.481481,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,6,2015,25,12,110011,0.480000,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,6,2014,23,11,107905,0.478261,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [36]:
combine['StoreType'].unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [37]:
combine.Assortment.unique()

array(['a', 'c', 'b'], dtype=object)

In [38]:
combine.loc[lambda x: np.logical_and(x.StoreType == 'c', x.Assortment == 'b')]

,Store,Month,Year,Open,Promo,Sales,PromoFrequencyRatio,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


In [39]:
combine.loc[lambda x: np.logical_and(x.StoreType == 'a', x.Assortment == 'b')]

,Store,Month,Year,Open,Promo,Sales,PromoFrequencyRatio,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


In [40]:
combine['StoreAssortment'] = combine['StoreType'] + combine['Assortment']

In [41]:
combine.StoreAssortment.unique()

array(['ca', 'aa', 'cc', 'ac', 'da', 'dc', 'ba', 'bb', 'bc'], dtype=object)

In [42]:
combine.to_csv('final3-2v2.csv', index=False)